In [1]:
!pip install --user tensorflow_hub
!pip3 install --user bert-for-tf2
!pip install --user bert-tensorflow


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
#import tokenization
import tensorflow as tf
import tensorflow_hub as hub
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [4]:
pip install --user protobuf==3.20.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from bert import bert_tokenization

In [5]:
#from textblob.en import polarity
import numpy as np
import pandas as pd
import tensorflow as tf
#from textblob import TextBlob
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xml.etree.ElementTree as ET
#import xmltodict

# # Reading the dataset into a dataframe
# from google.colab import drive
# drive.mount('/content/drive')



tree = ET.parse('../unprocessed.tar/sorted_data/sorted_data/books/negative.review')
root = tree.getroot()

data = []
for review in root:
    review_dict = {}
    for elem in review:
        if elem.tag in ['review_text', 'rating']:
            review_dict[elem.tag] = elem.text.strip()
        review_dict["polarity"] = "negative"
    data.append(review_dict)

tree = ET.parse('../unprocessed.tar/sorted_data/sorted_data/books/positive.review')
root = tree.getroot()
for review in root:
    review_dict = {}
    for elem in review:
        if elem.tag in ['review_text', 'rating']:
            review_dict[elem.tag] = elem.text.strip()
        review_dict["polarity"] = "positive"
    data.append(review_dict)

df = pd.DataFrame(data)
df['polarity_encoded'] = df['polarity'].apply(lambda x: 1 if x == 'positive' else 0)
print(df.head())
df

   polarity rating                                        review_text  \
0  negative    2.0  This book has its good points. If anything, it...   
1  negative    2.0  The fatalistic view of Conservativism is expre...   
2  negative    1.0  I was intrigued by the title, which supposedly...   
3  negative    2.0  I admit, I haven't finished this book.  A frie...   
4  negative    2.0  I tried reading this book but found it so turg...   

   polarity_encoded  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  


,polarity,rating,review_text,polarity_encoded
0,negative,2.0,"This book has its good points. If anything, it...",0
1,negative,2.0,The fatalistic view of Conservativism is expre...,0
2,negative,1.0,"I was intrigued by the title, which supposedly...",0
3,negative,2.0,"I admit, I haven't finished this book. A frie...",0
4,negative,2.0,I tried reading this book but found it so turg...,0
...,...,...,...,...
1995,positive,4.0,"As a boy, I was fascinated by looking at all o...",1
1996,positive,5.0,This book is as much about life as it is about...,1
1997,positive,5.0,If you are looking for ideas and techniques th...,1
1998,positive,5.0,This is a simple book to read through for youn...,1


In [6]:

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# print the shapes of the resulting DataFrames
print('Training data shape:', train_data.shape)
print('Testing data shape:', test_data.shape)


Training data shape: (1600, 4)
Testing data shape: (400, 4)


In [7]:
# label = preprocessing.LabelEncoder()
# y = label.fit_transform(train_data['polarity'])
# y = to_categorical(y)
y =  train_data['polarity_encoded']

In [8]:
#Build a BERT layer

m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(m_url, trainable=True)

In [10]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len-len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [11]:
max_len = 256
train_input = bert_encode(train_data.review_text.values, tokenizer, max_len=max_len)
test_input = bert_encode(test_data.review_text.values, tokenizer, max_len=max_len)
train_labels = y
test_labels = test_data['polarity_encoded']

In [15]:
def build_model_MLP(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    out = tf.keras.layers.Dense(1, activation='sigmoid')(pooled_output)

    threshold = tf.constant(0.5)
    def threshold_predictions(x):
        return tf.where(x > threshold, 1, 0)

    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    #model.metrics[-1].name = 'f1score'

    return model


In [16]:
model_MLP = build_model_MLP(bert_layer, max_len=max_len)
model_MLP.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 256)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 256)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 256, 768)]                'input_mask[0][0]',       

In [14]:
model_MLP = build_model_MLP(bert_layer, max_len=max_len)
model_MLP.summary()
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_sh = model_MLP.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    callbacks=[checkpoint, earlystopping],
    batch_size=8,
    verbose=1
)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 256)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 256)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 256, 768)]                'input_mask[0][0]',       

KeyboardInterrupt: 

In [12]:
# y_pred = np.where(y_pred >= 0.5, "positive", "negative")

test_labels = np.where(test_labels ==1, "positive", "negative")

In [14]:
from sklearn.metrics import classification_report

# Make predictions on the validation set
y_pred = model_MLP.predict(test_input)
y_pred = np.where(y_pred >= 0.5, "positive", "negative")

# Print the classification report
print(classification_report(test_labels, y_pred))

13/13 [==============================] - 2s 147ms/step
              precision    recall  f1-score   support

    negative       0.86      0.96      0.91       199
    positive       0.96      0.85      0.90       201

    accuracy                           0.90       400
   macro avg       0.91      0.90      0.90       400
weighted avg       0.91      0.90      0.90       400

